In [3]:
# using Pkg; Pkg.add("DataFrames"); Pkg.add("CSV")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed WeakRefStrings ─ v0.6.1
 Installed CSV ──────────── v0.5.11
  Updating `~/.julia/environments/v1.2/Project.toml`
  [336ed68f] + CSV v0.5.11
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [336ed68f] + CSV v0.5.11
  [ea10d353] + WeakRefStrings v0.6.1


In [1]:
using DataFrames

function df2mm(df::DataFrame)
    n = size(df, 1)
    mm_raw = [fill(1.0, n, 1)]
    mm_name = ["const"]
    for (name, value) in eachcol(df, true)
        if eltype(value) <: Real
            push!(mm_raw, hcat(Float64.(value)))
            push!(mm_name, string(name))
        else
            uvalue = unique(value)
            length(uvalue) == 1 && continue
            dvalue = Dict(v=>i for (i, v) in enumerate(uvalue))
            mvalue = zeros(n, length(uvalue))
            for i in 1:n
                mvalue[i, dvalue[value[i]]] = 1.0
            end
            push!(mm_raw, mvalue[:, 2:end])
            append!(mm_name, string.(name, "_", uvalue[2:end]))
        end
    end
    (data=hcat(mm_raw...), names=mm_name)
end

function lm(df, y, xs)
    yv = Float64.(df[!, y])
    xv, xn = df2mm(df[:, [xs;]])
    params = (transpose(xv)*xv)\(transpose(xv)*yv)
    DataFrame(name = xn, estimate=params)
end

lm (generic function with 1 method)

In [4]:
using CSV

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1242


In [5]:
wages = CSV.read("wages.csv", categorical=true);

In [6]:
lm(wages, :LWage, setdiff(names(wages), [:LWage]))

,name,estimate
,String,Float64
1,const,5.45403
2,Exp,0.0103709
3,Wks,0.00494447
4,BlueCol_yes,-0.148634
5,Ind,0.0530578
6,South_no,0.0532101
7,SMSA_yes,0.145304
8,Married_no,-0.0660799
9,Sex_female,-0.353302


In [7]:
function lm(df, y, xs)
    df = disallowmissing!(dropmissing!(df[:, [y; xs]]))
    yv = Float64.(df[!, y])
    xv, xn = df2mm(df[:, [xs;]])
    params = (transpose(xv)*xv)\(transpose(xv)*yv)
    DataFrame(name = xn, estimate=params)
end

lm (generic function with 1 method)

In [8]:
lm(wages, :LWage, setdiff(names(wages), [:LWage]))

,name,estimate
,String,Float64
1,const,5.45403
2,Exp,0.0103709
3,Wks,0.00494447
4,BlueCol_yes,-0.148634
5,Ind,0.0530578
6,South_no,0.0532101
7,SMSA_yes,0.145304
8,Married_no,-0.0660799
9,Sex_female,-0.353302
